# Thermal-Field emission vs Pure Field emission
The function FJgtf is from Kevins book and calculates the thermal-field emission using his method.

In [3]:
using PyPlot
#import PhysicalConstants.CODATA2018: ħ, m_e, e, ε_0

In [4]:
# Variables
rmo = 5.685629853;
rkb = 1.0/11604.5192;
rhbar = 0.658211899;
alpha = 7.29735257E-3;
c = 299.7924580;
epso = 1.0/(4*pi*alpha*rhbar*c);
Qo = alpha*rhbar*c/4;
Ampcm2 = 1.602176565E10;
Arld = (rmo*rkb^2)/(2*rhbar*(pi*rhbar)^2);

In [5]:
function FJgtf(Phi, Chem, Fld, Tmp)
    betaT = 1.0/(rkb*Tmp)
    y = sqrt(4*Qo*Fld)/Phi
    phix = Phi - sqrt(4*Qo*Fld)
    vy = 1.0 - (3.0 - log(y))*(y^2)/3
    ty = 1.0 + (1.0 - log(y))*(y^2)/9
    Qmu = 4*sqrt(2*rmo*Phi^3)*vy/(3*rhbar*Fld)
    Bfmu = 2*sqrt(2*rmo*Phi)*ty/( rhbar*Fld)
    Bfmphi = pi*sqrt(y*rmo*Phi) /( rhbar*Fld)
    Tmin = (rhbar*Fld/(4*ty*rkb))*sqrt(2.0/(rmo*Phi))
    Tmax = (rhbar*Fld/(pi*rkb)) *sqrt(1.0/(y*rmo*Phi))
    
    # Three regimes:
    if(Tmp < Tmin) #Field Emission Regime
        s = Qmu
        rn = betaT/Bfmu
    elseif (Tmp > Tmax) then #Thermal Emission Regime
        s = Bfmphi*Phix
        rn = betaT/BFmphi
    else #Intermediate Regime
        rn = 1.0
        Ap = 3*(phix*(Bfmu + Bfmphi) - 2*Qmu)
        Bp = 2*(3*Qmu - phix*(2*Bfmu + Bfmphi))
        Cp = phix*(Bfmu - BetaT)
        p = (-Bp-sqrt(Bp^2 - 4*Ap*Cp))/(2*Ap)
        Em = Chem + p*phix
        s = betaT*(Em - Chem + (ThetaE(Phi,Chem,Fld,Em)/BetaE(Phi,Chem,Fld,Em)))
    end
            
    RJgtf = (Arld/(rkb*betaT)^2)*RNns(rn,s)*Ampcm2
end

function ThetaE(Phi, Chem, Field, E)
    y = sqrt(4*Qo*Field)/Phi
    phix = Phi - sqrt(4*Qo*Field)
    vy = 1.0 - (3.0 - log(y))*(y^2)/3
    ty = 1.0 + (1.0 - log(y))*(y^2)/9
    Qmu = 4*sqrt(2*rmo*Phi^3)*vy/(3*rhbar*Field)
    Bfmu = 2*sqrt(2*rmo*Phi)*ty/( rhbar*Field)
    Bfmphi = pi*sqrt(y*rmo*Phi) /( rhbar*Field)
    p = (E - Chem)/phix
    ThetaE = Qmu*(2*p+1.0)*(1.0 - p)^2 - phix*Bfmu*p*(1.0 - p)^2 + phix*Bfmphi*(1.0 - p)*p^2
end

function BetaE(Phi, Chem, Field, E)
    y = sqrt(4*Qo*Field)/Phi
    phix = Phi - sqrt(4*Qo*Field)
    vy = 1.0 - (3.0 - log(y))*(y^2)/3
    ty = 1.0 + (1.0 - log(y))*(y^2)/9
    Qmu = 4*sqrt(2*rmo*Phi^3)*vy/(3*rhbar*Field)
    Bfmu = 2*sqrt(2*rmo*Phi)*ty/( rhbar*Field)
    Bfmphi = pi*sqrt(y*rmo*Phi) /( rhbar*Field)
    p = (E - Chem)/phix
    BetaE = (6*Qmu/phix)*p*(1.0 - p) - Bfmu*(1.0 - p)*(3*p - 1.0) + Bfmphi*p*(3*p - 2.0)
end

function RNns(n, s)
    x = n^2
    y = 1.0/x
    sn = -x*(0.10593434*x + 0.35506593)
    sd = -y*(0.10593434*y + 0.35506593)
    z = (n-1.0)*s
    if(abs(z) > 0.001)
        sng = (n^2 + 1.0)*((n^2)*exp(-s) - exp(-n*s))/(n^2 - 1.0)
    else
        sng = (0.5*(n^2 + 1.0)*exp(-s)/(n+1.0))* ((1.0-n)*(s^2) + 2*(1.0+n) + 2*s)
    end
    RNns = sng + sn*exp(-n*s) + x*sd*exp(-s)
end

RNns (generic function with 1 method)

In [6]:
Phi = 4.5 # Work function [eV]
Chem = 7.0 # Chemical potential [eV] (This number dosen't seem to affect the calculations in any way)
F = 2.0 # Field [eV/nm] [GV/m]
T = 0.01 # Temperature [K] 
FJgtf(Phi, Chem, F, T)

0.00041577168700286063

Call this function to convert to and from the units Kevin uses</br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Field: V/m -> eV/nm</br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Current density: A/cm² -> A/m²</br>
Note that in Kevins units the electron charge is e = 1. This means that 1 V = 1 eV and that 1 GV/m = 1 eV/nm. See Chapter 2 page 10 in Kevins book Introduction to the Physics of Electrom Emission.

In [7]:
function Get_Kevin_Jgtf(F, T, w_theta)
    F_evnm = abs(F) * 1.0E-9 # Kevin wants the field in eV/nm see note above (Basically we convert to GV/m).
    Get_Kevin_Jgtf = FJgtf(w_theta, Chem, F_evnm, T) * 1.0E4 # Kevin returns in A/cm² convert to A/m²
end

Get_Kevin_Jgtf (generic function with 1 method)

# Fowler-Nordheim
Define the Fowler-Nordheim function

In [8]:
hbar = 1.054571817E-34
e = 1.602176634E-19
m_e = 9.1093837015E-31
epsilon_0 = 8.8541878128E-12

#------------------------------------------------------------------------------------------
a_FN = e^2/(16.0*pi^2*hbar) # A eV V^{-2}
b_FN = -4.0/(3.0*hbar) * sqrt(2.0*m_e*e) # eV^{-3/2} V m^{-1}
l_const = e / (4.0*pi*epsilon_0) # eV^{2} V^{-1} m

function t_y(F, w_theta)
    l = l_const * F / w_theta^2 # l = y^2, y = 3.79E-4 * sqrt(F_old) / w_theta
    t_y = 1.0 + l*( 1.0/9.0 - 1.0/18.0*log(l) )
end

function v_y(F, w_theta)
    l = l_const * F / w_theta^2 # l = y^2, y = 3.79E-4 * sqrt(F_old) / w_theta
    v_y = 1.0 - l + 1.0/6.0 * l * log(l)
end

# ----------------------------------------------------------------
# Calculates the Fowler-Nordheim current density
# F the field in V/m
# w_theta the work function in eV
function J_FN(F_, w_theta)
    F = abs(F_)
    J_FN = a_FN/(w_theta*t_y(F, w_theta)^2)*F^2 * exp(v_y(F, w_theta)*b_FN*w_theta^(3/2)/F)
end

J_FN (generic function with 1 method)

# System
Define system test parameters

In [9]:
V_0 = 2.0E3 # Voltage
d = 1000.0E-9
w_theta = 2.0
T = 100.0
time_step = 1.0E-4*1.0E-12
L = 1000.0E-9
A = L^2

F = V_0/d

2.0e9

In [10]:
Get_Kevin_Jgtf(F, T, w_theta)

2.5441102800184567e11

In [11]:
J_FN(F, w_theta)

2.5246632401237506e11

In [12]:
Chem = 7.0
FJgtf(w_theta, Chem, F, T)

UndefVarError: UndefVarError: BetaT not defined

In [13]:
J_FN(F, w_theta)*time_step/e*A

157.577085231899

In [14]:
Get_Kevin_Jgtf(F, T, 2.0)

2.5441102800184567e11

In [15]:
Get_Kevin_Jgtf(F, T, 2.5)

3.34022947639468e9

In [16]:
Get_Kevin_Jgtf(F, T, 2.0)/Get_Kevin_Jgtf(F, T, 2.5)

76.16573346225526

In [17]:
Get_Kevin_Jgtf(F, T, 2.5)/Get_Kevin_Jgtf(F, T, 2.0)

0.013129263706172548

In [18]:
(exp(v_y(F, 2.5)*b_FN*w_theta^(3/2)/F))/(exp(v_y(F, 2.0)*b_FN*w_theta^(3/2)/F))

0.09927122065668276